In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

from lmmnn.nn import reg_nn_ohe_or_ignore, reg_nn_lmm, reg_nn_embed, reg_nn_menet
from lmmnn.simulation import Count

In [3]:
# Note: data_cleaned_train_comments_X.csv is the result of an ETL process described in Kalehbasti et. al. (2019), see our paper.
# We followed the script in their Github repo exactly.
path = 'C:/Users/gsimchoni/AirBnbPricePrediction/Data/'
X_train = pd.read_csv(path + 'data_cleaned_train_comments_X.csv')
y_train = pd.read_csv(path + 'data_cleaned_train_y.csv').values
y_train = y_train.reshape(len(y_train), )

X_val = pd.read_csv(path + 'data_cleaned_val_comments_X.csv')
y_val = pd.read_csv(path + 'data_cleaned_val_y.csv').values
y_val = y_val.reshape(len(y_val), )

X_test = pd.read_csv(path + 'data_cleaned_test_comments_X.csv')
y_test = pd.read_csv(path + 'data_cleaned_test_y.csv').values
y_test = y_test.reshape(len(y_test), )

In [4]:
coeffs = np.load(path + 'selected_coefs.npy')
col_set = set()

for i in range(len(coeffs)):
        if (coeffs[i]):
            col_set.add(X_train.columns[i])
X_train = X_train[list(col_set | set(['host_id']))]
X_val = X_val[list(col_set | set(['host_id']))]
X_test = X_test[list(col_set | set(['host_id']))]

X = pd.concat([X_train, X_val, X_test], ignore_index=True)
y = np.concatenate([y_train, y_val, y_test])

In [5]:
print(X.shape)
print(y.shape)

(49976, 197)
(49976,)


In [6]:
# relevant Adam params taken from original paper
NUM_ITERATIONS = 1000
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DECAY_RATE = 0.0001

# original paper Adam configuration:
# adam = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=DECAY_RATE, amsgrad=False)

# some other params
mode = 'intercepts'
n_sig2bs = 1
est_cors = []
n_neurons = [20, 5]
dropout = None
activation = 'relu'

In [7]:
n_cats = max([X_train['host_id'].max(), X_val['host_id'].max(), X_test['host_id'].max()]) + 1

In [8]:
len(col_set)

196

In [9]:
col_set

{'24-hour_check-in',
 'Accessible-height_bed',
 'Accessible-height_toilet',
 'Air_conditioning',
 'Apartment',
 'Arverne.1',
 'Astoria.1',
 'Baby_bath',
 'Babysitter_recommendations',
 'Bathtub',
 'Bay Ridge',
 'Bed_linens',
 'Bensonhurst',
 'Boerum Hill',
 'Borough Park',
 'Boutique hotel',
 'Breakfast',
 'Bronx',
 'Bronx.1',
 'Brooklyn ',
 'Brooklyn Heights',
 'Brooklyn.1',
 'Bushwick.1',
 'Buzzer/wireless_intercom',
 'Cable_TV',
 'Carbon_monoxide_detector',
 'Carroll Gardens',
 'Cat(s)',
 'Chelsea',
 'Children’s_books_and_toys',
 'Clinton Hill',
 'Cobble Hill',
 'Coffee_maker',
 'Cooking_basics',
 'Crib',
 'Cypress Hills',
 'DUMBO',
 'Dishes_and_silverware',
 'Dishwasher',
 'Doorman',
 'Downtown Brooklyn',
 'Dryer',
 'East Flatbush',
 'East Harlem',
 'East Village',
 'Elevator',
 'Elmhurst.1',
 'Entire home/apt',
 'Essentials',
 'Ethernet_connection',
 'Extra_pillows_and_blankets',
 'Family/kid_friendly',
 'Financial District',
 'Fire_extinguisher',
 'First_aid_kit',
 'Flat_path_to_

In [10]:
X.rename(columns={'host_id': 'z0'}, inplace=True)

In [11]:
def reg_nn(X_train, X_test, y_train, y_test, n_cats, batch=30, epochs=100, patience=10, reg_type='ohe', verbose=False):
    start = time.time()
    if reg_type == 'ohe':
        y_pred, sigmas, _, n_epochs = reg_nn_ohe_or_ignore(X_train, X_test, y_train, y_test, [n_cats], x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, mode, n_sig2bs, est_cors, verbose)
    elif reg_type == 'lmm':
        y_pred, sigmas, _, n_epochs = reg_nn_lmm(X_train, X_test, y_train, y_test, [n_cats], x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, mode, n_sig2bs, est_cors, None, verbose)
    elif reg_type == 'ignore':
        y_pred, sigmas, _, n_epochs = reg_nn_ohe_or_ignore(X_train, X_test, y_train, y_test, [n_cats], x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, mode, n_sig2bs, est_cors, verbose, ignore_RE=True)
    elif reg_type == 'embed':
        y_pred, sigmas, _, n_epochs = reg_nn_embed(X_train, X_test, y_train, y_test, [n_cats], x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, mode, n_sig2bs, est_cors, verbose)
    elif reg_type == 'menet':
        y_pred, sigmas, _, n_epochs = reg_nn_menet(X_train, X_test, y_train, y_test, n_cats, x_cols, batch, epochs, patience,
            n_neurons, dropout, activation, n_sig2bs, est_cors, verbose)
    else:
      raise ValueError(reg_type + 'is an unknown reg_type')
    end = time.time()
    mse = np.mean((y_pred - y_test)**2)
    return mse, sigmas, n_epochs, end - start

In [12]:
res = pd.DataFrame(columns=['experiment', 'exp_type', 'mse', 'sigma_e_est', 'sigma_b0_est', 'n_epoch', 'time'])
kf = KFold(n_splits=5, shuffle=True, random_state=40)
# kf = KFold(n_splits=5)
counter = Count().gen()
x_cols = [col for col in X.columns if col != 'z0']

In [13]:
def iterate_reg_types(X_train, X_test, y_train, y_test, counter, verbose):
    mse_lmm, sigmas, n_epochs_lmm, time_lmm = reg_nn(X_train, X_test, y_train, y_test, n_cats, reg_type='lmm', verbose=verbose)
    print(' finished lmm, mse: %.2f' % mse_lmm)
    mse_ohe, _, n_epochs_ohe, time_ohe = reg_nn(X_train, X_test, y_train, y_test, n_cats, reg_type='ohe', verbose=verbose)
    print(' finished ohe, mse: %.2f' % mse_ohe)
    mse_ig, _, n_epochs_ig, time_ig = reg_nn(X_train, X_test, y_train, y_test, n_cats, reg_type='ignore', verbose=verbose)
    print(' finished ignore, mse: %.2f' % mse_ig)
    mse_em, _, n_epochs_em, time_em = reg_nn(X_train, X_test, y_train, y_test, n_cats, reg_type='embed', verbose=verbose)
    print(' finished embed, mse: %.2f' % mse_em)
    mse_me, sigmas_me, n_epochs_me, time_me = reg_nn(X_train, X_test, y_train, y_test, n_cats, reg_type='menet', verbose=verbose)
    print(' finished menet, mse: %.2f' % mse_me)
    res.loc[next(counter)] = [i, 'lmm', mse_lmm, sigmas[0], sigmas[1][0], n_epochs_lmm, time_lmm]
    res.loc[next(counter)] = [i, 'ohe', mse_ohe, np.nan, np.nan, n_epochs_ohe, time_ohe]
    res.loc[next(counter)] = [i, 'ignore', mse_ig, np.nan, np.nan, n_epochs_ig, time_ig]
    res.loc[next(counter)] = [i, 'embed', mse_em, np.nan, np.nan, n_epochs_em, time_em]
    res.loc[next(counter)] = [i, 'menet', mse_me, sigmas_me[0], np.nan, n_epochs_me, time_me]

In [15]:
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    print('iteration %d' % i)
    X_train, X_test, y_train, y_test = X.loc[train_index], X.loc[test_index], y[train_index], y[test_index]
    # scaler = StandardScaler()
    # y_train = scaler.fit_transform(y_train.values.reshape(-1, 1)).reshape(X_train.shape[0])
    # y_test = scaler.transform(y_test.values.reshape(-1, 1)).reshape(X_test.shape[0])
    y_train = pd.Series(y_train, index=X_train.index)
    y_test = pd.Series(y_test, index=X_test.index)
    iterate_reg_types(X_train, X_test, y_train, y_test, counter, verbose=True)

In [17]:
res

In [ ]:
res.to_csv('../results/res_airbnb.csv')